In [1]:
print("Ok")

Ok


In [2]:
%pwd

'c:\\My Area\\Projects\\GenAI_Learning\\Nedical Chatbot\\Nedical_Chatbot-with-LLMs-LangChain-Pinecone-Flask-Aws\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\My Area\\Projects\\GenAI_Learning\\Nedical Chatbot\\Nedical_Chatbot-with-LLMs-LangChain-Pinecone-Flask-Aws'

In [5]:
# from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

c:\Users\abhis\anaconda3\envs\nedibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Extract text from PDF Files
def load_pdf_files(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [7]:
extracted_data = load_pdf_files("data")

In [8]:
# extracted_data

In [9]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    Containing only 'source' in metadata and the orginal page_content."""
    minimal_docs: list[Document] = []
    for doc in docs:
        src= doc.metadata.get("source")
        minimal_docs.append(
            Document(page_content=doc.page_content, metadata={"source": src})
        )
    return minimal_docs

In [10]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [11]:
# minimal_docs

In [12]:
# split the documents into smaller chunks

def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20
    )
    text_chunks = text_splitter.split_documents(minimal_docs)
    return text_chunks

In [13]:
text_chunks = text_split(minimal_docs)
print(f"Number of text chunks: {len(text_chunks)}")

Number of text chunks: 5859


In [14]:
# from langchain.embeddings import HuggingFaceEmbeddings   OLD
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download & return HuggingFace Embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings
embedding = download_embeddings()

C:\Users\abhis\AppData\Local\Temp\ipykernel_2252\1908732076.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)


In [16]:
vector = embedding.embed_query("Hello world")
# vector

In [22]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [23]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [24]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [25]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [26]:
pc

In [27]:
from pinecone import ServerlessSpec
index_name = "nedical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

index = pc.Index(index_name)

In [28]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embedding,
    index_name=index_name
)

In [29]:
# Load Exisiting Index
from langchain_pinecone import PineconeVectorStore
# embed each chunk and insert  the embeddings into pinecone index
docsearch = PineconeVectorStore.from_existing_index(
     index_name=index_name,
     embedding=embedding
)

## Add More Data to the Existing PineCone Index

In [31]:
dswith = Document(
    page_content="DswithAnkesh is a youtube channel that provides tutorials on programming, web development, and data science.",
    metadata={"source": "Youtube"}
)

In [32]:
docsearch.add_documents([dswith])

['c03b6b11-153f-4593-af29-a868c3f92be6']

In [33]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [34]:
retrieved_docs = retriever.invoke("What is Acne")
retrieved_docs

[Document(id='1d81927c-985f-48ed-bd1e-07ace361a79c', metadata={'source': 'data\\Medical_book (1).pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='fea7f785-52e1-4a24-87e3-7c3bb3b766b1', metadata={'source': 'data\\Medical_book (1).pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is\nthe most common skin disease. It affects nearly 17 million\npeople in the United States. While acne can arise at any'),
 Document(id='66ca4e83-5ebc-42c2-84b7-4007b9ec50af', metadata={'source': 'data\\Medical_book (1).pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne

In [35]:
from dotenv import load_dotenv
import os
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [36]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize Gemini model
LLM = ChatGoogleGenerativeAI(
    google_api_key=GOOGLE_API_KEY, 
    model="gemini-2.5-flash", 
    temperature=0
)

c:\Users\abhis\anaconda3\envs\nedibot\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [37]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

In [38]:
system_prompt = (
    "You are medcial assistant for  question_answering task"
"Use the following pieces of retrevieved context to answer"
"the question. If you don't know the answer, just say that you don't know, don't try to make up an answer."
"Use three sentences maximum to answer the question & keep the answer concise"
"\n\n"
"{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
    )

In [39]:
question_answering_chain = create_stuff_documents_chain(LLM, prompt)
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [40]:
response = rag_chain.invoke({"input": "What is acromegaly & gigantisim?"})
print(response["answer"])

Acromegaly is a disorder where the pituitary gland in the brain abnormally releases a chemical, leading to increased growth in bone and soft tissue. This condition also causes a variety of other disturbances throughout the body. The provided text discusses acromegaly and gigantism together, but only offers a direct definition for acromegaly.
